In [10]:
import torch.quantization
import torch
import os


model_path='./retinanet.pt'
    
myModel = torch.load(model_path,map_location='cpu')
quantized_model = torch.quantization.quantize_dynamic(myModel, {torch.nn.Linear}, dtype=torch.qint8)
myModel = torch.quantization.get_default_config('fbgemm')

# # insert observers
torch.quantization.prepare(myModel, inplace=True)
# # Calibrate the model and collect statistics

# # convert to quantized version
torch.quantization.convert(myModel, inplace=True)
print(quantized_model)
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)
torch.save(quantized_model, 'model_lite.pt')


AttributeError: module 'torch.quantization' has no attribute 'get_default_config'

In [16]:
import torch.quantization
import torch
import os
num_calibration_batches = 32

model_path='./retinanet.pt'
    
myModel = torch.load(model_path,map_location='cpu')
myModel.eval()

# Fuse Conv, bn and relu
# myModel.fuse_model()

# Specify quantization configuration
# Start with simple min/max range estimation and per-tensor quantization of weights
myModel.qconfig = torch.quantization.default_qconfig
print(myModel.qconfig)
torch.quantization.prepare(myModel, inplace=True)

# Calibrate first
print('Post Training Quantization Prepare: Inserting Observers')
# print('\n Inverted Residual Block:After observer insertion \n\n', myModel.features[1].conv)

# Calibrate with the training set
# evaluate(myModel, criterion, data_loader, neval_batches=num_calibration_batches)
# print('Post Training Quantization: Calibration done')

# Convert to quantized model
torch.quantization.convert(myModel, inplace=True)
print('Post Training Quantization: Convert done')
# print('\n Inverted Residual Block: After fusion and quantization, note fused modules: \n\n',myModel.features[1].conv)

print("Size of model after quantization")
print_size_of_model(myModel)
torch.save(myModel, 'model_lite.pt')
# top1, top5 = evaluate(myModel, criterion, data_loader_test, neval_batches=num_eval_batches)
# print('Evaluation accuracy on %d images, %2.2f'%(num_eval_batches * eval_batch_size, top1.avg))

QConfig(activation=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, reduce_range=True){}, weight=functools.partial(<class 'torch.ao.quantization.observer.MinMaxObserver'>, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric){})
Post Training Quantization Prepare: Inserting Observers


/home/thinh-do/.local/lib/python3.8/site-packages/torch/ao/quantization/observer.py:172: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
/home/thinh-do/.local/lib/python3.8/site-packages/torch/ao/quantization/utils.py:156: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


Post Training Quantization: Convert done
Size of model after quantization
Size (MB): 36.911009
